In [31]:
import torch
from torch import nn,optim
import functools
import matplotlib.pyplot as plt
from torch.autograd import Variable
import torchvision.utils as vutils

In [32]:
import random
import numpy as np
import torch.utils.data as data
from PIL import Image
import torchvision.transforms as transforms
from skimage.transform import resize

In [33]:
from tensorboardX import SummaryWriter
writer=SummaryWriter('/user1/faculty/cvpr/ujjwal/dhritimaan/GANs/datasets/log_cycle_gan_ssim/')

In [34]:
class SequenceFolder(data.Dataset):
    def __init__(self, trainA,trainB):
        self.trainA=trainA
        self.trainB=trainB

    def __len__(self):
        return 1556
    def __getitem__(self, index):
        index_a=index%len(self.trainA)
        index_b=index%len(self.trainB)
        imgA=plt.imread(self.trainA[index_a])
        imgB=plt.imread(self.trainB[index_b])
        imgA=resize(imgA,(256,256,3))
        imgB=resize(imgB,(256,256,3))
        transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))])
        #index=index%
        dict_img={'A':imgA,'B':imgB}
        return dict_img

In [35]:
#plt.imread(foggy_paths[0]).shape
import os

In [36]:
file1 = open("/user1/faculty/cvpr/ujjwal/kaiser/dataset/foggy_zurich/Foggy_Zurich/lists_file_names/RGB_medium_filenames.txt",
             "r")
x=file1.read()
foggy_paths=[os.path.join('/user1/faculty/cvpr/ujjwal/kaiser/dataset/foggy_zurich/Foggy_Zurich',t) for t in x.splitlines()]

In [37]:
import os
city=os.listdir('/user1/faculty/cvpr/ujjwal/kaiser/dataset/cityscape/leftImg8bit/train')

In [38]:
img_paths=[]
for i in range(10):
    city_path=os.path.join('/user1/faculty/cvpr/ujjwal/kaiser/dataset/cityscape/leftImg8bit/train',city[i])
    img_names=os.listdir(city_path)
    k=[os.path.join(city_path,s) for s in img_names]
    for j in range(len(k)):
        img_paths.append(k[j])

In [39]:
#foggy_paths

In [40]:
train=SequenceFolder(foggy_paths,img_paths)#lista,listb)

In [41]:
dataloader=torch.utils.data.DataLoader(train,batch_size=3,shuffle=True)

In [42]:
#import matplotlib.pyplot as plt
#for a,b in enumerate(dataloader):
#    a
#    break

In [43]:
#b['A'].shape

In [44]:
#b['B'].permute(0,3,1,2).shape

In [45]:
#torch.cat((b['A'].permute(0,3,1,2),b['B'].permute(0,3,1,2)),1).shape

In [46]:
class ResnetBlock(nn.Module):
    """Define a Resnet block"""

    def __init__(self, dim, padding_type, norm_layer, use_dropout, use_bias):
        """Initialize the Resnet block
        A resnet block is a conv block with skip connections
        We construct a conv block with build_conv_block function,
        and implement skip connections in <forward> function.
        Original Resnet paper: https://arxiv.org/pdf/1512.03385.pdf
        """
        super(ResnetBlock, self).__init__()
        self.conv_block = self.build_conv_block(dim, padding_type, norm_layer, use_dropout, use_bias)

    def build_conv_block(self, dim, padding_type, norm_layer, use_dropout, use_bias):
        """Construct a convolutional block.
        Parameters:
            dim (int)           -- the number of channels in the conv layer.
            padding_type (str)  -- the name of padding layer: reflect | replicate | zero
            norm_layer          -- normalization layer
            use_dropout (bool)  -- if use dropout layers.
            use_bias (bool)     -- if the conv layer uses bias or not
        Returns a conv block (with a conv layer, a normalization layer, and a non-linearity layer (ReLU))
        """
        conv_block = []
        p = 0
        if padding_type == 'reflect':
            conv_block += [nn.ReflectionPad2d(1)]
        elif padding_type == 'replicate':
            conv_block += [nn.ReplicationPad2d(1)]
        elif padding_type == 'zero':
            p = 1
        else:
            raise NotImplementedError('padding [%s] is not implemented' % padding_type)

        conv_block += [nn.Conv2d(dim, dim, kernel_size=3, padding=p, bias=use_bias), norm_layer(dim), nn.ReLU(True)]
        if use_dropout:
            conv_block += [nn.Dropout(0.5)]

        p = 0
        if padding_type == 'reflect':
            conv_block += [nn.ReflectionPad2d(1)]
        elif padding_type == 'replicate':
            conv_block += [nn.ReplicationPad2d(1)]
        elif padding_type == 'zero':
            p = 1
        else:
            raise NotImplementedError('padding [%s] is not implemented' % padding_type)
        conv_block += [nn.Conv2d(dim, dim, kernel_size=3, padding=p, bias=use_bias), norm_layer(dim)]

        return nn.Sequential(*conv_block)

    def forward(self, x):
        """Forward function (with skip connections)"""
        out = x + self.conv_block(x)  # add skip connections
        return out

In [47]:
class ResnetGenerator(nn.Module):
    """Resnet-based generator that consists of Resnet blocks between a few downsampling/upsampling operations.
    We adapt Torch code and idea from Justin Johnson's neural style transfer project(https://github.com/jcjohnson/fast-neural-style)
    """

    def __init__(self, input_nc, output_nc, ngf=64, norm_layer=nn.BatchNorm2d, use_dropout=False, n_blocks=6, padding_type='reflect'):
        """Construct a Resnet-based generator
        Parameters:
            input_nc (int)      -- the number of channels in input images
            output_nc (int)     -- the number of channels in output images
            ngf (int)           -- the number of filters in the last conv layer
            norm_layer          -- normalization layer
            use_dropout (bool)  -- if use dropout layers
            n_blocks (int)      -- the number of ResNet blocks
            padding_type (str)  -- the name of padding layer in conv layers: reflect | replicate | zero
        """
        assert(n_blocks >= 0)
        super(ResnetGenerator, self).__init__()
        if type(norm_layer) == functools.partial:
            use_bias = norm_layer.func == nn.InstanceNorm2d
        else:
            use_bias = norm_layer == nn.InstanceNorm2d

        model = [nn.ReflectionPad2d(3),
                 nn.Conv2d(input_nc, ngf, kernel_size=7, padding=0, bias=use_bias),
                 norm_layer(ngf),
                 nn.ReLU(True)]

        n_downsampling = 2
        for i in range(n_downsampling):  # add downsampling layers
            mult = 2 ** i
            model += [nn.Conv2d(ngf * mult, ngf * mult * 2, kernel_size=3, stride=2, padding=1, bias=use_bias),
                      norm_layer(ngf * mult * 2),
                      nn.ReLU(True)]

        mult = 2 ** n_downsampling
        for i in range(n_blocks):       # add ResNet blocks

            model += [ResnetBlock(ngf * mult, padding_type=padding_type, norm_layer=norm_layer, use_dropout=use_dropout, use_bias=use_bias)]

        for i in range(n_downsampling):  # add upsampling layers
            mult = 2 ** (n_downsampling - i)
            model += [nn.ConvTranspose2d(ngf * mult, int(ngf * mult / 2),
                                         kernel_size=3, stride=2,
                                         padding=1, output_padding=1,
                                         bias=use_bias),
                      norm_layer(int(ngf * mult / 2)),
                      nn.ReLU(True)]
        model += [nn.ReflectionPad2d(3)]
        model += [nn.Conv2d(ngf, output_nc, kernel_size=7, padding=0)]
        model += [nn.Tanh()]

        self.model = nn.Sequential(*model)

    def forward(self, input):
        """Standard forward"""
        return self.model(input)

In [48]:
class NLayerDiscriminator(nn.Module):
    """Defines a PatchGAN discriminator"""

    def __init__(self, input_nc, ndf=64, n_layers=3, norm_layer=nn.BatchNorm2d):
        """Construct a PatchGAN discriminator
        Parameters:
            input_nc (int)  -- the number of channels in input images
            ndf (int)       -- the number of filters in the last conv layer
            n_layers (int)  -- the number of conv layers in the discriminator
            norm_layer      -- normalization layer
        """
        super(NLayerDiscriminator, self).__init__()
        if type(norm_layer) == functools.partial:  # no need to use bias as BatchNorm2d has affine parameters
            use_bias = norm_layer.func == nn.InstanceNorm2d
        else:
            use_bias = norm_layer == nn.InstanceNorm2d

        kw = 4
        padw = 1
        sequence = [nn.Conv2d(input_nc, ndf, kernel_size=kw, stride=2, padding=padw), nn.LeakyReLU(0.2, True)]
        nf_mult = 1
        nf_mult_prev = 1
        for n in range(1, n_layers):  # gradually increase the number of filters
            nf_mult_prev = nf_mult
            nf_mult = min(2 ** n, 8)
            sequence += [
                nn.Conv2d(ndf * nf_mult_prev, ndf * nf_mult, kernel_size=kw, stride=2, padding=padw, bias=use_bias),
                norm_layer(ndf * nf_mult),
                nn.LeakyReLU(0.2, True)
            ]

        nf_mult_prev = nf_mult
        nf_mult = min(2 ** n_layers, 8)
        sequence += [
            nn.Conv2d(ndf * nf_mult_prev, ndf * nf_mult, kernel_size=kw, stride=1, padding=padw, bias=use_bias),
            norm_layer(ndf * nf_mult),
            nn.LeakyReLU(0.2, True)
        ]

        sequence += [nn.Conv2d(ndf * nf_mult, 1, kernel_size=kw, stride=1, padding=padw)]  # output 1 channel prediction map
        self.model = nn.Sequential(*sequence)

    def forward(self, input):
        """Standard forward."""
        return self.model(input)

In [49]:
cuda=torch.device('cuda:1')
gen_encoder_a=ResnetGenerator(3,6).to(cuda)
gen_encoder_b=ResnetGenerator(3,6).to(cuda)
gen_common=ResnetGenerator(6,3).to(cuda)
gen_style_ba=ResnetGenerator(3,3).to(cuda)
#gen_style_ba=ResnetGenerator(3,3).to(cuda)

In [50]:
#disc_style=NLayerDiscriminator(3).to(cuda)
#disc_a=NLayerDiscriminator(3).to(cuda)
#disc_b=NLayerDiscriminator(3).to(cuda)
disc_style_ab=NLayerDiscriminator(3).to(cuda)
disc_style_ba=NLayerDiscriminator(3).to(cuda)

In [51]:
from itertools import chain

In [52]:
gen_optim=optim.Adam(chain(gen_encoder_a.parameters(),gen_encoder_b.parameters(),
                     gen_common.parameters(),gen_style_ba.parameters())#,gen_style_ab.parameters())
                     ,lr=0.0002,betas=(0.5,0.999))
dis_optim=optim.Adam(chain(#disc_a.parameters(),disc_b.parameters(),
                           disc_style_ab.parameters(),
                           disc_style_ba.parameters()),lr=0.0002,betas=(0.5,0.999))

In [53]:
#loss_ce=nn.NLLLoss()
loss_mse=nn.MSELoss()
#loss_rec=l1loss()+-

In [54]:
def l1loss(x,y):
    return torch.mean(torch.abs(x-y))
def mse(x,y):
    return torch.mean((x-y)**2)

In [55]:
#nn.L1Loss()
#import skimage
#gen_encoder(torch.zeros(1,3,256,256).to(cuda)).shape

In [56]:
from skimage.transform import resize

In [57]:
#genAB(torch.cat((b['A'],b['A']),1).permute(0,3,1,2).float().to(cuda)/255)

# Train for interchanged content and style

In [58]:
import pytorch_ssim
ssim_loss=pytorch_ssim.SSIM()

In [59]:
def normalize(x):
    if x < 0:
        return 1-x
    else:
        return x

In [60]:
num_epochs=100
count=0
for epoch in range(num_epochs):
    for n_batches,real_data in enumerate(dataloader):
        foggy=real_data['A'].permute(0,3,1,2).float().to(cuda)#/255
        normal=real_data['B'].permute(0,3,1,2).float().to(cuda)#/255
        
        #realA=torch.cat((realA_,realA_),1)
        #realB=torch.cat((realA_,realB_),1)
        
        #GENERATE STYLE, CONTENT
        st_ct_a=gen_encoder_a(foggy)
        st_ct_b=gen_encoder_b(normal)
        content_a=st_ct_a[:,:3,:,:]
        style_a=st_ct_a[:,3:,:,:]
        content_b=st_ct_b[:,:3,:,:]
        style_b=st_ct_b[:,3:,:,:]
        #GENERATE FAKE AND REC STYLE USING STYLE_GEN
        fake_style_b=gen_style_ba(style_a)
        fake_style_a=gen_style_ba(style_b)
        
        
        rec_style_a=gen_style_ba(fake_style_b)
        rec_style_b=gen_style_ba(fake_style_a)
        
        #CREATE REC IMAGES AND INTER CONTENT AND STYLE IMAGES
        rec_a=gen_common(torch.cat((content_a,style_a),1))
        rec_b=gen_common(torch.cat((content_b,style_b),1))
        cbsa=gen_common(torch.cat((content_b,style_a),1))
        casb=gen_common(torch.cat((content_a,style_b),1))
        cbsa_fake=gen_common(torch.cat((content_b,fake_style_a),1))
        casb_fake=gen_common(torch.cat((content_a,fake_style_b),1))
        #cacb=gen_common(torch.cat(content_a,content_b),1)
        #sasb=gen_common(torch.cat(style_a,style_b),1)
        
        # GERNERATE LABELS DISC FOR REAL IMAGES
#       label_rec_b=disc_b(rec_b)
#       label_real_b=disc_b(normal)
#       label_rec_a=disc_a(rec_a)
#       label_real_a=disc_a(foggy)
          #GEN LABELS FOR STYLE CYCLE GAN
        label_style_a=disc_style_ba(fake_style_a)
        label_style_b=disc_style_ab(fake_style_b)
        
        gen_optim.zero_grad()
        #######training generators
        #LOSS IMAGES
#         error_disc_b=loss_mse(label_rec_b,torch.ones(foggy.shape[0],1,30,30).to(cuda))
#         error_dis_a=loss_mse(label_rec_a,torch.ones(foggy.shape[0],1,30,30).to(cuda))
        rec_errorA=l1loss(rec_a,foggy)#loss_rec(recA,realA)
        rec_errorB=l1loss(rec_b,normal)#loss_rec(recB,realB)
        #LOSS STYLE
        error_style_a=loss_mse(label_style_a,torch.ones(foggy.shape[0],1,30,30).to(cuda))
        error_style_b=loss_mse(label_style_b,torch.ones(foggy.shape[0],1,30,30).to(cuda))
        
        ssim_content_a=ssim_loss(content_a,foggy)
        ssim_content_b=ssim_loss(content_b,normal)
        
        
        
        #ssim_style_a=ssim_loss(style_a,foggy)
        #ssim_style_b=ssim_loss(style_b,normal)
        #str_loss_b=l1loss(rec_b,normal)
        #str_loss_a=l1loss(rec_a,foggy)
        
        ssim_content_loss=10*ssim_content_a+ssim_content_b
        #ssim_style_loss=10*ssim_style_a+ssim_style_b   
        #STYLE REC LOSS
        rec_style_a=l1loss(rec_style_a,style_a)
        rec_style_b=l1loss(rec_style_b,style_b)
        
        rec_style_loss=rec_style_a+rec_style_b
        
        g_loss=10*rec_errorA+10*rec_errorB-(ssim_content_loss)+10*rec_style_loss
        #error_genAB.backward(retain_graph=True)
        #error_genBA.backward()
        #rec_errorA.backward()
        #rec_errorB.backward()
        g_loss.backward()#retain_graph=True)
        
        gen_optim.step()
    
        #########training discrminators
        dis_optim.zero_grad()
        
        st_ct_a=gen_encoder_a(foggy)
        st_ct_b=gen_encoder_b(normal)
        content_a=st_ct_a[:,:3,:,:]
        style_a=st_ct_a[:,3:,:,:]
        content_b=st_ct_b[:,:3,:,:]
        style_b=st_ct_b[:,3:,:,:]
        
        fake_style_b=gen_style_ba(style_a)
        fake_style_a=gen_style_ba(style_b)
        rec_style_a=gen_style_ba(fake_style_b)
        rec_style_b=gen_style_ba(fake_style_a)
        
        rec_a=gen_common(torch.cat((content_a,style_a),1))
        rec_b=gen_common(torch.cat((content_b,style_b),1))
        
#         label_rec_b=disc_b(rec_b)
#         label_real_b=disc_b(normal)
#         label_rec_a=disc_a(rec_a)
#         label_real_a=disc_a(foggy)
        
        label_style_b=disc_style_ab(fake_style_b)
        label_style_a=disc_style_ba(fake_style_a)
        label_style_realb=disc_style_ab(style_b)
        label_style_reala=disc_style_ba(style_a)
        
#         error__rec_disc_b=loss_mse(label_rec_b,Variable(torch.zeros(foggy.shape[0],1,30,30).to(cuda)))
#         error__real_disc_b=loss_mse(label_real_b,Variable(torch.ones(foggy.shape[0],1,30,30).to(cuda)))
#         error__rec_disc_a=loss_mse(label_rec_a,Variable(torch.zeros(foggy.shape[0],1,30,30).to(cuda)))
#         error__real_disc_a=loss_mse(label_real_a,Variable(torch.ones(foggy.shape[0],1,30,30).to(cuda)))
        error__real_disc_style_a=loss_mse(label_style_reala,Variable(torch.ones(foggy.shape[0],1,30,30).to(cuda)))
        error__fake_disc_style_a=loss_mse(label_style_a,Variable(torch.zeros(foggy.shape[0],1,30,30).to(cuda)))
        error__real_disc_style_b=loss_mse(label_style_realb,Variable(torch.ones(foggy.shape[0],1,30,30).to(cuda)))
        error__fake_disc_style_b=loss_mse(label_style_b,Variable(torch.zeros(foggy.shape[0],1,30,30).to(cuda)))
        
#         disc_b_loss=(error__rec_disc_b+error__real_disc_b)/2
#         disc_a_loss=(error__rec_disc_a+error__real_disc_a)/2
        disc_style_a_loss=(error__real_disc_style_a+error__fake_disc_style_a)/2
        disc_style_b_loss=(error__real_disc_style_b+error__fake_disc_style_b)/2
        disc_loss=disc_style_a_loss+disc_style_b_loss
        disc_loss.backward()
        #error_discAB_with_fakeB.backward()
        #error_discBA_with_realB.backward()
        #error_discBA_with_realB.backward()
        dis_optim.step()
        
        
        if count%10==0:
            writer.add_image("rec_a_",vutils.make_grid(rec_a,padding=2,normalize=True),count)#foggy
            writer.add_image("rec_b_",vutils.make_grid(rec_b,padding=2,normalize=True),count)##city
            writer.add_image("cbsa",vutils.make_grid(gen_common(torch.cat((content_b,style_a),1)),
                                                     padding=2,normalize=True),count)
            writer.add_image("casb",vutils.make_grid(gen_common(torch.cat((content_a,style_b),1)),
                                                     padding=2,normalize=True),count)
            writer.add_image("cbsa_fake",vutils.make_grid(gen_common(torch.cat((content_b,fake_style_a),1)),
                                                     padding=2,normalize=True),count)
            writer.add_image("casb_fake",vutils.make_grid(gen_common(torch.cat((content_a,fake_style_b),1)),
                                                     padding=2,normalize=True),count)
            writer.add_image("foggy",vutils.make_grid(foggy,padding=2,normalize=True),count)
            writer.add_image("normal",vutils.make_grid(normal,padding=2,normalize=True),count)
            
            writer.add_image("content_a",vutils.make_grid(content_a,padding=2,normalize=True),count)
            writer.add_image("style_a",vutils.make_grid(style_a,padding=2,normalize=True),count)
            writer.add_image("content_b",vutils.make_grid(content_b,padding=2,normalize=True),count)
            writer.add_image("style_b",vutils.make_grid(style_b,padding=2,normalize=True),count)
            
            writer.add_scalar("disc_foggy",disc_style_b_loss.item(),count)
            writer.add_scalar("disc_normal",disc_style_a_loss.item(),count)
            
            writer.add_scalar("recon_label_b",label_style_b.mean(),count)
            writer.add_scalar("recon_label_a",label_style_a.mean(),count)
            
            writer.add_scalar("disc_loss",disc_loss.item(),count)
            writer.add_scalar("gen_loss",g_loss.item(),count)
        
        
        count=count+1
        print("epoch:{}  disc_loss:{}  gen_loss:{} ssim_content_a:{} ssim_content_b:{} ssim_style_a:{} ssim_style_b:{}".format(epoch
                                    ,disc_loss,g_loss,ssim_content_a,ssim_content_b,disc_style_a_loss,
                                disc_style_b_loss))

epoch:0  disc_loss:1.4162472486495972  gen_loss:18.540424346923828 ssim_content_a:0.003545601386576891 ssim_content_b:0.010400990955531597 ssim_style_a:0.8287664651870728 ssim_style_b:0.5874807834625244
epoch:0  disc_loss:6.867903232574463  gen_loss:12.975445747375488 ssim_content_a:0.01481380220502615 ssim_content_b:0.011114249937236309 ssim_style_a:3.562892198562622 ssim_style_b:3.305011034011841
epoch:0  disc_loss:4.854732513427734  gen_loss:9.292856216430664 ssim_content_a:0.031273771077394485 ssim_content_b:0.04548889398574829 ssim_style_a:2.781578540802002 ssim_style_b:2.0731542110443115
epoch:0  disc_loss:0.8638041615486145  gen_loss:5.512343883514404 ssim_content_a:0.05343766510486603 ssim_content_b:0.08767519146203995 ssim_style_a:0.41615378856658936 ssim_style_b:0.44765037298202515
epoch:0  disc_loss:0.8331135511398315  gen_loss:4.094754219055176 ssim_content_a:0.0809093564748764 ssim_content_b:0.164889857172966 ssim_style_a:0.41381412744522095 ssim_style_b:0.4192994236946106

epoch:0  disc_loss:0.15164104104042053  gen_loss:-2.7614994049072266 ssim_content_a:0.4080501198768616 ssim_content_b:0.592671275138855 ssim_style_a:0.07496106624603271 ssim_style_b:0.07667997479438782
epoch:0  disc_loss:0.12415013462305069  gen_loss:-2.964174747467041 ssim_content_a:0.38595083355903625 ssim_content_b:0.6633709073066711 ssim_style_a:0.06691797077655792 ssim_style_b:0.05723216384649277
epoch:0  disc_loss:0.12145848572254181  gen_loss:-2.3215579986572266 ssim_content_a:0.3846096098423004 ssim_content_b:0.5892149806022644 ssim_style_a:0.08789774030447006 ssim_style_b:0.033560749143362045
epoch:0  disc_loss:0.2653234601020813  gen_loss:-2.716336727142334 ssim_content_a:0.3829498291015625 ssim_content_b:0.6085063815116882 ssim_style_a:0.11440729349851608 ssim_style_b:0.15091615915298462
epoch:0  disc_loss:0.19006246328353882  gen_loss:-2.7461299896240234 ssim_content_a:0.42437735199928284 ssim_content_b:0.6560676693916321 ssim_style_a:0.09035610407590866 ssim_style_b:0.0997

epoch:0  disc_loss:0.14753012359142303  gen_loss:-3.3611416816711426 ssim_content_a:0.4277159869670868 ssim_content_b:0.7189591526985168 ssim_style_a:0.06545064598321915 ssim_style_b:0.08207947760820389
epoch:0  disc_loss:0.07912640273571014  gen_loss:-3.394115924835205 ssim_content_a:0.45164188742637634 ssim_content_b:0.7236673831939697 ssim_style_a:0.04720218852162361 ssim_style_b:0.031924210488796234
epoch:0  disc_loss:0.057249005883932114  gen_loss:-4.034132480621338 ssim_content_a:0.4668767750263214 ssim_content_b:0.7940981984138489 ssim_style_a:0.04095514491200447 ssim_style_b:0.016293860971927643
epoch:0  disc_loss:0.1417616605758667  gen_loss:-3.723788261413574 ssim_content_a:0.4364253878593445 ssim_content_b:0.7814318537712097 ssim_style_a:0.09201608598232269 ssim_style_b:0.049745574593544006
epoch:0  disc_loss:0.09435635060071945  gen_loss:-3.84673810005188 ssim_content_a:0.45303788781166077 ssim_content_b:0.7803698182106018 ssim_style_a:0.031079242005944252 ssim_style_b:0.06

epoch:0  disc_loss:0.010535920038819313  gen_loss:-4.226466655731201 ssim_content_a:0.46686989068984985 ssim_content_b:0.8029435276985168 ssim_style_a:0.004818161018192768 ssim_style_b:0.005717759020626545
epoch:0  disc_loss:0.013352788984775543  gen_loss:-4.291249752044678 ssim_content_a:0.4580426812171936 ssim_content_b:0.826920747756958 ssim_style_a:0.005485564935952425 ssim_style_b:0.007867223583161831
epoch:0  disc_loss:0.05983597785234451  gen_loss:-3.881033182144165 ssim_content_a:0.4867525100708008 ssim_content_b:0.7213475108146667 ssim_style_a:0.04200268164277077 ssim_style_b:0.017833296209573746
epoch:0  disc_loss:0.011554405093193054  gen_loss:-4.228851795196533 ssim_content_a:0.4695119857788086 ssim_content_b:0.7850465774536133 ssim_style_a:0.0053477841429412365 ssim_style_b:0.006206621415913105
epoch:0  disc_loss:0.028593799099326134  gen_loss:-4.6858906745910645 ssim_content_a:0.5007228851318359 ssim_content_b:0.807618260383606 ssim_style_a:0.01948891207575798 ssim_style_

epoch:0  disc_loss:0.07378758490085602  gen_loss:-4.032846450805664 ssim_content_a:0.4964626133441925 ssim_content_b:0.7509442567825317 ssim_style_a:0.03251553699374199 ssim_style_b:0.04127204418182373
epoch:0  disc_loss:0.14299409091472626  gen_loss:-4.03966760635376 ssim_content_a:0.4768179953098297 ssim_content_b:0.7913821339607239 ssim_style_a:0.07372623682022095 ssim_style_b:0.06926785409450531
epoch:0  disc_loss:0.17082718014717102  gen_loss:-3.9824109077453613 ssim_content_a:0.4412299692630768 ssim_content_b:0.8588879704475403 ssim_style_a:0.09265769273042679 ssim_style_b:0.07816949486732483
epoch:0  disc_loss:0.0765339657664299  gen_loss:-4.190791130065918 ssim_content_a:0.4551866948604584 ssim_content_b:0.8296573162078857 ssim_style_a:0.04423699527978897 ssim_style_b:0.03229697048664093
epoch:0  disc_loss:0.03417184576392174  gen_loss:-4.3580803871154785 ssim_content_a:0.47708258032798767 ssim_content_b:0.8133991360664368 ssim_style_a:0.020702747628092766 ssim_style_b:0.013469

...

epoch:10  disc_loss:0.014883574098348618  gen_loss:-5.934017658233643 ssim_content_a:0.5620703101158142 ssim_content_b:0.9537222385406494 ssim_style_a:0.004914113786071539 ssim_style_b:0.009969459846615791
epoch:10  disc_loss:0.0058660046197474  gen_loss:-5.6780171394348145 ssim_content_a:0.5203259587287903 ssim_content_b:0.9499784111976624 ssim_style_a:0.002087114378809929 ssim_style_b:0.0037788902409374714
epoch:10  disc_loss:0.12411762028932571  gen_loss:-5.735518932342529 ssim_content_a:0.5477269291877747 ssim_content_b:0.9480627179145813 ssim_style_a:0.027822211384773254 ssim_style_b:0.09629540890455246
epoch:10  disc_loss:0.09008466452360153  gen_loss:-5.6518707275390625 ssim_content_a:0.5346754789352417 ssim_content_b:0.9657171368598938 ssim_style_a:0.024404117837548256 ssim_style_b:0.06568054854869843
epoch:10  disc_loss:0.07442976534366608  gen_loss:-5.530287742614746 ssim_content_a:0.5478439927101135 ssim_content_b:0.9430060982704163 ssim_style_a:0.01621360518038273 ssim_styl

epoch:10  disc_loss:0.013607414439320564  gen_loss:-5.757565498352051 ssim_content_a:0.5364487171173096 ssim_content_b:0.9546520709991455 ssim_style_a:0.002096914453431964 ssim_style_b:0.011510499753057957
epoch:10  disc_loss:0.020870596170425415  gen_loss:-6.233896255493164 ssim_content_a:0.5636494755744934 ssim_content_b:0.9660347700119019 ssim_style_a:0.003952488768845797 ssim_style_b:0.016918107867240906
epoch:10  disc_loss:0.008051984012126923  gen_loss:-5.6047210693359375 ssim_content_a:0.5262998342514038 ssim_content_b:0.9677194952964783 ssim_style_a:0.0036741779185831547 ssim_style_b:0.004377806559205055
epoch:10  disc_loss:0.003281788667663932  gen_loss:-6.300789833068848 ssim_content_a:0.5898976922035217 ssim_content_b:0.9509463906288147 ssim_style_a:0.0014757635071873665 ssim_style_b:0.0018060251604765654
epoch:10  disc_loss:0.001445010770112276  gen_loss:-6.1232500076293945 ssim_content_a:0.5605910420417786 ssim_content_b:0.9632234573364258 ssim_style_a:0.000476715213153511

epoch:10  disc_loss:0.05674855411052704  gen_loss:-5.970709800720215 ssim_content_a:0.557693600654602 ssim_content_b:0.9642737507820129 ssim_style_a:0.01445676013827324 ssim_style_b:0.0422917939722538
epoch:10  disc_loss:0.06900110840797424  gen_loss:-5.943115234375 ssim_content_a:0.5410389304161072 ssim_content_b:0.968993067741394 ssim_style_a:0.012642123736441135 ssim_style_b:0.05635898560285568
epoch:10  disc_loss:0.006230402272194624  gen_loss:-6.118486404418945 ssim_content_a:0.5608391165733337 ssim_content_b:0.948573887348175 ssim_style_a:0.0028517101891338825 ssim_style_b:0.0033786920830607414
epoch:10  disc_loss:0.01901411823928356  gen_loss:-6.086085796356201 ssim_content_a:0.5790680646896362 ssim_content_b:0.9534088969230652 ssim_style_a:0.001171383890323341 ssim_style_b:0.0178427342325449
epoch:10  disc_loss:0.20466181635856628  gen_loss:-6.80838680267334 ssim_content_a:0.6723154187202454 ssim_content_b:0.9435995221138 ssim_style_a:0.09055584669113159 ssim_style_b:0.11410597

epoch:10  disc_loss:0.02801956795156002  gen_loss:-6.154362678527832 ssim_content_a:0.5745171904563904 ssim_content_b:0.9643166065216064 ssim_style_a:0.004572046454995871 ssim_style_b:0.023447521030902863
epoch:10  disc_loss:0.004535764455795288  gen_loss:-5.801172733306885 ssim_content_a:0.5385043025016785 ssim_content_b:0.948358952999115 ssim_style_a:0.002517262240871787 ssim_style_b:0.0020185019820928574
epoch:10  disc_loss:0.007291831076145172  gen_loss:-5.611380577087402 ssim_content_a:0.5068060755729675 ssim_content_b:0.9535312652587891 ssim_style_a:0.0010489923879504204 ssim_style_b:0.006242838688194752
epoch:10  disc_loss:0.004907915368676186  gen_loss:-5.920568943023682 ssim_content_a:0.5449841618537903 ssim_content_b:0.96142578125 ssim_style_a:0.001061916584149003 ssim_style_b:0.003845998551696539
epoch:10  disc_loss:0.005594633519649506  gen_loss:-5.68221378326416 ssim_content_a:0.5272529721260071 ssim_content_b:0.9690081477165222 ssim_style_a:0.0012401520507410169 ssim_styl

KeyboardInterrupt: 

In [34]:
torch.tensor(6.0)

tensor(6.)

In [34]:
torch.f(4).sqrt()

RuntimeError: sqrt_vml_cpu not implemented for 'Long'

In [32]:
!nvidia-smi

Tue Dec 24 21:27:51 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.129      Driver Version: 410.129      CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P6            On   | 00000000:17:00.0 Off |                    0 |
| N/A   38C    P0    84W /  90W |  14231MiB / 15261MiB |     98%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P6            On   | 00000000:45:00.0 Off |                    0 |
| N/A   36C    P0    64W /  90W |  15248MiB / 15261MiB |    100%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T